# Agent with Tool Use
One of the benefits of an agent is its ability to use Tools. Leveraging Tools enables agents to perform complex tasks beyond reasoning capabilities.
When given a task, the agent evaluates the task requirements, then selects an appropriate Tool from its available toolkit, and strategically applies them to achieve specific goals.

This process involves using the LLM to reason about tool selection, understanding how to properly utilize each Tool, and integrating the results back into the agent's workflow.
In addition, agents could also be given access to external knowledge bases, interface with APIs to extend their problem-solving capability that can address a broader range of challenges.

In this lab, we are going to walk through an example of an agent with Tool use.
This use case focuses on answering a user's questions about media assets. Media asset details, such as titles, plot, synopsis are types of information commonly stored and available in the title management systems of media companies. These details could be leveraged by an AI agent to gather the information about the titles based on a user's questions in natural language. While title information is stored in the knowledge base, there may be additional details relevant to titles that are not readily available, for instances, getting the latest movie ratings or reviews about the given title. An agent would need to access other internal / external sources, for instance, IMDB through web APIs to gather such details.

In our lab, we demonstrate calling an API through a Tool that allows the agent to gather extra information. The combination of details from knowledge bases and the API completes the user query.

<img src="../../imgs/agent-tooluse-architecture.png" width="600"> 

The steps shown in the above diagram are defined as followed:

1. User initiates a task request. (e.g. Ask for a rating of a movie)
2. An agent received the request, first extracts the media title information from the knowledge base.
3. Based on the user query, the agent retrieves the rating information by invoking a Tool. The Tool makes an API {all that provides the rating information for a title given the title_id.
4. The agent receives both the title and rating information for the given title.
5. Agent returns the final response to the user.

Let's get started!

Restore variables for context to be used in this notebook

In [2]:
%store -r

In [3]:
import boto3
import os
import uuid
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.anthropic.claude-3-5-haiku-20241022-v1:0'
]

## Importing helper functions
In the following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, Knowledge Bases for Bedrock creation, and accessing data in S3.

In [4]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")
sys.path.insert(2, "../..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock, upload_directory
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

# Creating an Agent

Create a media agent that will have access to an `Amazon Bedrock Knowledge Base` where it retrieves the given title information.

The role of the agent is to answer user questions about movie titles, including synopsis, rating and more.

We will provide the following instructions to our agent:
```
You are a professional media agent. Your task is to help users find the information
related to the media based on the tool and knowledge bases available to you.

You have access to a knowledge base and tools that provide information about media details, including the title, and title_id.
You also have access to tools that can help provide additional information about the title, such as rating and others.
```

We will also connect a knowledge base to retrieve good PR references. Follow the instructions as described below:
```
This knowledge base contains information about media titles. Use this knowledge base to find information about the media, including the title and id.
```

In [5]:
kb_info = kb.get_kb(lab6_kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

Create a configuration for the Knowledge Base. The configuration is to be used in the agent creation process.

In [6]:
kb_config = {
    'kb_id': lab6_kb_id,
    'kb_instruction': """This knowledge base contains information about media titles. Use this knowledge base to find information about the media, including the title and id."""
}

In [7]:
agent_description = """A media agent that helps users find media information."""
agent_instruction = """You are a professional media agent. Your task is to help users find the information
related to the media based on the tool and knowledge bases available to you.

You have access to a knowledge base and tools that provide information about media details, including the title, and title_id.
You also have access to tools that can help provide additional information about the title, such as rating.

If you need to retrieve the title id from the knowledge base, look for the title_id column for the value.
For example, a retrieved media contains the following data:

title_id: 123
title: Some title
year: 2025
duration: 100 minutes

Based on the example above, you should retrieve the title_id of 123.
The title_id should not be retrieved from the <source></source> XML tag.
"""

Create a Bedrock Agent using the agent helper function 

In [ ]:
agent_suffix = str(uuid.uuid4())[:5]
agent_name = f"lab6-media_agent-{agent_suffix}"

media_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(20) # giving some time for the agent to be in a ready state.
media_agent

### Associating Knowledge Base
Now that we've created the agent, let's associate the previously created Knowledge Base to it.

In [9]:
agents.associate_kb_with_agent(
    media_agent[0], # the agent ID
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

# Define Agent Tools
In the context of Bedrock Agents, Tools are organized as Action Groups. An Action Group defines actions that the agent can help the user perform. For example, you could define an action group called `Get the title rating via an API call` that helps users get the ratings of a given title.

You create an Action Group by performing the following steps:

Define the parameters and information that the agent must elicit from the user for each action in the Action Group to be carried out.

Decide how the agent handles the parameters and information that it receives from the user and where it sends the information it elicits from the user.

To learn more about Action Groups, please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html).

Each Action Group supports functions that help the agent to fulfill a subtask. For instance, a function that invokes an external API to retrieve the rating of a title, or an API that retrieves the most recent reviews about a title.

The function details consist of a list of parameters, defined by their name, data type (for a list of supported data types, see ParameterDetail), and whether they are required. The agent uses these configurations to determine what information it needs to elicit from the user. You can define the function detail in a JSON file with name, description, parameters, or provide a file in the OpenAPI compatible format.

To fulfill the task, you can define a Lambda function to program the business logic for an Action Group. After a Amazon Bedrock agent determines the API operation that it needs to invoke in an Action Group, it sends information from the API schema alongside relevant metadata as an input event to the Lambda function. To write your function, you must understand the following components of the Lambda function:

Input event – Contains relevant metadata and populated fields from the request body of the API operation or the function parameters for the action that the agent determines must be called.

Response – Contains relevant metadata and populated fields for the response body returned from the API operation or the function.

For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html)

If a Lambda function is not feasible, another option is to choose to return control to the agent developer by sending the information in the InvokeAgent response. For more information please refer to this [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-returncontrol.html)

In this lab, we'll define an Action Group with a Lambda function that simulates an API call. The API returns the rating of a given title id, which is obtained from the knowledge base that the agent has access.

In [10]:
action_group_name=f"get-media-ratings-{agent_suffix}"
action_group_descr = "Get the title rating via an API call"
lambda_code = "lambda_handler.py"
function_defs = [
    {
          "name": "get_title_rating",
          "description": "Get the title rating via an API call",
          "parameters": {
            "title_id": {
              "description": "title_id for a given title as extracted from the knowledge bases",
              "required": True,
              "type": "string"
            }
          },
          "requireConfirmation": "DISABLED"
    }
        ]

Creat an Action Group with lambda function

In [11]:
lambda_function_name = f"lab6-get-media-ratings-{agent_suffix}"
action_group_arn = agents.add_action_group_with_lambda(agent_name,
                                         lambda_function_name, lambda_code, 
                                         function_defs, action_group_name, action_group_descr)

# Testing the Agent
With all the components in place, let's test out our agent. We'll start by asking some questions about specific titles. 

In [ ]:
response = agents.invoke(
    """Tell me the genre of "Quantum Shadows".""", 
    media_agent[0], enable_trace=True,
    trace_level = "all"
)
print("====================")
print(response)

In [ ]:
%%time
response = agents.invoke(
    """What is the rating this movie? 
""", 
    media_agent[0], enable_trace=True,
    trace_level = "all"
)
print("====================")
print(response)

# Conclusion
In this lab, we demonstrated how to build a Bedrock Agent as an AI assistant that helps users find media information efficiently. We first created a Knowledge Base for Bedrock, which contains the media details made available to the agent to find media title information. Additionally, we also created a Tool for the agent which provides real time media rating from an external system via an API call. Using these tools, the AI agent helps users answer relevant questions about the media assets in their media library / archive. In the next lab, we'll extend the agentic capabilities to incorporate multiple AI agents to work collaboratively to solve an even more complex problem. Go ahead and hop over to lab 7 to continue the workshop.